### 微博评论获取
#### 郑宇浩 41621101
#### 该文件主要对应于论文中的注释[10]和[11]，实现2014年3月3日至2016年2月29日所有带关键词“股票市场”、“股票代码”和“股票交易”的原创微博获取及其情感评分

In [1]:
import datetime
import pandas as pd
import numpy as np
import requests
import re
import jieba
import time as t
import os
path=os.path.abspath('.')

#### 1. 数据结构设计：构建DataFrame的索引为2014年3月3日到2016年2月29日每日早晨7点至午夜12点每隔2小时为一单位的时间点

In [2]:
date_start=datetime.datetime.strptime("2014-03-03",'%Y-%m-%d')
date_end=datetime.datetime.strptime("2016-02-29",'%Y-%m-%d')
date=[]
date.append(date_start.strftime('%Y-%m-%d'))
while date_start<date_end:
    date_start+=datetime.timedelta(days=+1)
    date.append(date_start.strftime('%Y-%m-%d'))

time=['7','9','11','13','15','17','19','21','23']
day=len(date)
hour=len(time)-1

Day=np.repeat(date,hour)
Hour=time[:-1]*day
df=pd.DataFrame({'Day':Day,'Hour':Hour})

#### 2. 微博评论获取：仅展示前2天的评论获取，请勿一次性获取729天，因为每5天代码会自动休息10分钟，这会耗费你一天的时间去获取完整的评论

In [3]:
Raw=[]
for i in range(2):
    for j in range(hour):
        url=r'https://s.weibo.com/weibo/%25E8%2582%25A1%25E7%25A5%25A8%25E4%25BA%25A4%25E6%2598%2593?q=%E8%82%A1%E7%A5%A8%E4%BA%A4%E6%98%93&scope=ori&suball=1&timescope=custom:'+date[i]+'-'+time[j]+':'+date[i]+'-'+time[j+1] # 股票交易
        # url=r'https://s.weibo.com/weibo/%25E8%2582%25A1%25E7%25A5%25A8%25E4%25BB%25A3%25E7%25A0%2581?q=%E8%82%A1%E7%A5%A8%E4%BB%A3%E7%A0%81&scope=ori&suball=1&timescope=custom:'+date[i]+'-'+time[j]+':'+date[i]+'-'+time[j+1] # 股票代码
        # url=r'https://s.weibo.com/weibo?q=%E8%82%A1%E7%A5%A8%E5%B8%82%E5%9C%BA&scope=ori&suball=1&timescope=custom:'+date[i]+'-'+time[j]+':'+date[i]+'-'+time[j+1] # 股票市场
        r=requests.get(url)
        s=r.text
        Raw.append(s)
    if((i+1)%5==0): # 由于微博访问次数的限制，每抓取5天的评论需要休息10分钟
        t.sleep(600)

#### 3. 数据清洗与情感分析：请务必先将所需文件Upload到jupyter中，文件位于与该notebook同目录下的文件夹，文件名为long.txt、short.txt、否定词.txt，评分公式请参考论文中的公式(4)

In [4]:
from string import punctuation as punc
punc=punc+" 、，；：‘’“”—。？！……【】（）《》－"
re_tag=re.compile(r'<(.*?)>')

def comment_clean(text): # 去除标点符号与特殊字符串
    clean=re_tag.sub('',text)
    clean="".join(w for w in clean if w not in punc)
    return clean

positive=open(path+"\long.txt","r").read().split()
negative=open(path+"\short.txt","r").read().split()
deny=open(path+"\否定词.txt","r").read().split()

def get_score(clean): # 对清洗过后的评论进行情感评分
    clean=list(jieba.cut(clean))
    pos_count=len([w for w in clean if w in positive])
    neg_count=len([w for w in clean if w in negative])
    de_count=len([w for w in clean if w in deny])
    if pos_count==0:
        if neg_count==0: # 若正负向情感词都为0，则无情感，为0分
            score=0
        else: # 若负向情感词不为0，则评分-1
            score=-1
    else:
        if neg_count==0: # 若正向情感词不为0，则评分1
            score=1
        else: # 若正负向情感均不为0，则根据二者比值判断情感得分
            langda=pos_count/neg_count
            if langda>1.5: # 正向词数量比负向词数量大于1.5，则评分1
                score=1
            elif 1/langda>1.5: # 负向词数量比正向词数量大于1.5，则评分1
                score=-1
            else:
                score=0
    if de_count%2!=0:
        score=0-score
    return score

#### 4. 情感评分

In [5]:
Comment=[]
Score=[]
n=len(Raw)
for i in range(n):
    s=Raw[i] # 获取网页原始信息
    pattern=r">\n                    (.*?)\u200b"
    result=re.findall(pattern,s) # 获取所有评论
    if(result): # 若当前时间段内有评论，则记录其内容与情感得分
        clean_comment=[]
        emotion_score=[]
        for k in result:
            clean_comment.append(comment_clean(k)) # 数据清洗
        for k in clean_comment:
            emotion_score.append(get_score(k)) # 评论情感评分
        Comment.append(clean_comment)
        Score.append(emotion_score)
    else:
        Comment.append([])
        Score.append([])

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\PC\AppData\Local\Temp\jieba.cache
Loading model cost 1.495 seconds.
Prefix dict has been built succesfully.


In [6]:
Comment

[['现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '现货黄金黄金td纸黄金价格走势分析今日黄金价格黄金走势分析现货白银的优势价格上涨您做多赚价格下跌你做空也赚股票涨才赚跌则亏双向交易真正的上涨下跌都赚钱更多黄金白银投资详情加李哲老师QQ328389238天通银软件',
  '宁贵银指导天通银投资股票不好做涨1天跌4天没关系我们不做股票现货黄金买涨买跌都可以24小时交易这一秒买进下一秒获利就可以收钱还有专业老师为您全程指教更多投资理财分享群327510229验证婷婷老师无论您是做短线中线还是长线都可以给您最完美的建议',
  '白银走势白银价格现货白银操作建议白银分析纸白银价格走势富就富在不知足贵就贵在能脱俗贫就贫在少见识贱就贱在没骨气人生需要理财股票已过时双向连续交易可止损止盈人民币波动89元

In [7]:
Score

[[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 1, 1],
 [0, 0, 0, 1, 0, 0, 0, 0, -1, 0, -1, 1],
 [1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, 0, 0, 0, 0, 0],
 [1, 0, -1, -1, 0, 0, -1, 1, 1, -1, 1, 0, 0, 0, 1, 1, -1],
 [0, 0, -1, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [-1, 0, -1, -1, -1, 0, 0, 0, -1, 0, 0, -1, 0, 0, 0],
 [0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, -1, -1, 0, 0, -1, 1, 0, -1, 0, 0, 0, -1, 0],
 [0, 1, 0, 0, 0, -1, 0, -1],
 [0, 0, 0, 1, 0, 0, 0, -1, 0, 0, 0, 1, 0, 0],
 [0, 0, -1, 1, 1],
 [-1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 1, 0, -1],
 [0, 0, 0, 0, -1, -1, 1, 0, 0],
 [0, 0, 0, -1, 1, 0, 1, 1, 0, 0, 0],
 [-1, -1, 0, 0, 1, 0, 0, 0, 0, 0]]

#### 5. 数据导出

In [8]:
n=len(Comment)
new_df=pd.DataFrame({'Day':Day[:n],'Hour':Hour[:n],'comment':Comment,'score':Score})
new_df.to_excel(path+"\原始评论情感得分.xlsx",index=False)